In [6]:
from dataclasses import dataclass
import time
from tqdm import tqdm
from multiprocessing import Pool
from sklearn import preprocessing

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import obspy as op
from obspy import read,read_inventory, UTCDateTime, Stream, Trace
from obspy.clients.fdsn.client import Client
from obspy.signal.rotate import rotate_ne_rt
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel

import json
import glob
import os
import numpy as np
from itertools import combinations
import pandas as pd
from scipy.signal import spectrogram, detrend, resample,savgol_filter,decimate,hilbert
from scipy.stats import circmean, circstd

import pyarrow.feather as feather
import seaborn as sns

import datetime

____________
# Setup
____________

In [3]:
# ===========
# DIRECTORIES
# ===========

## ------------------------
## Directory with waveforms (SeisComP Data Structure)
## The basic directory and file layout is defined as:
## <SDSdir>/Year/NET/STA/CHAN.TYPE/NET.STA.LOC.CHAN.TYPE.YEAR.DAY

WAVEFORM_DIR = '/medata03/SEISCOMP_DATA/'

## ---------------------------
## Directory with the catalog (.CSV file of the National Earthquake Information Center (NEIC))
## The file layout is defined as:
## time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource

CATALOG_FILE = '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/CATALOG/catalog_2010_2025.csv'

## ------------------------
## Directory of the catalog (.CSV file of the National Earthquake Information Center (NEIC))
## The file layout is defined as:
## time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource

XML_DIR = '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/'

## -----------------------
## Directory of the output (Figures and Feathers file)

ORIENTATION_OUTPUT = '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/OUTPUT/'

# ==========
# PARAMETERS
# ==========

## -------------------------------------------------------------------
## Apply band-pass filtering to the seismograms using the range above:

PERIOD_BANDS = [0.02,0.5]

## ----------------------------------
## Define the time range for analysis:

FIRSTDAY = '2010-01-01'  # Start date (YYYY-MM-DD)  
LASTDAY = '2025-12-31'   # End date (YYYY-MM-DD) 

## ===================================================================================
## Default parameters to define the signal and noise windows used to estimate the SNR:

## ------------------------------------------------------------------------------
## Duration of the signal window before and after the P-wave arrival (in seconds)

TIME_WINDOW = 60

## -----------------------------------------------------------------------------
## Start time of the P-wave window for events (in seconds before P-wave arrival)

TIME_START_P_REGIONAL = 3

## -----------------------------------------------------------------------------------
## End time of the P-wave window for regional events (in seconds after P-wave arrival)

TIME_FINAL_P_REGIONAL = 12

## ---------------------------------------------
## Minimum earthquake magnitude to be considered

minmagnitude = 6

## -------------------------------------------------------------------------------------
## Minimum and maximum epicentral distance in degrees (GCARC: great-circle arc distance)

GCARC_MIN = 5
GCARC_MAX = 100


## -----------------
## Region parameters

LLCRNRLON_LARGE = -50
URCRNRLON_LARGE = -38
LLCRNRLAT_LARGE = -30
URCRNRLAT_LARGE = -12

## ---------
## Constants

ONEDAY = datetime.timedelta(days=1)

## ---------------
## MULTIPROCESSING

num_processes = 20

## --------------------------------------
## Generate a figure for each estimation?

VERBOSE = True

---
# Filtering by date

In [4]:
fday = UTCDateTime(FIRSTDAY)
lday = UTCDateTime(LASTDAY)
INTERVAL_PERIOD = [UTCDateTime(x.astype(str)) for x in np.arange(fday.datetime,lday.datetime+ONEDAY,ONEDAY)]
INTERVAL_PERIOD_DATE = [str(x.year)+'.'+"%03d" % x.julday for x in INTERVAL_PERIOD]

In [8]:
# =========
# Functions
# =========

def rms(x):
    """
    Function to calculate root-mean-square of array

    Parameters
    ----------
    x : :class:`~numpy.ndarray`
        Input array

    Returns
    -------
    rms : float
        Root-Mean-Square value of `x`
    """

    return np.sqrt(np.mean(x**2))

def energy(x):
    """
    Function to calculate energy of array

    Parameters
    ----------
    x : :class:`~numpy.ndarray`
        Input array

    Returns
    -------
    energy : float
        Square value of `x`
    """

    return np.sum(x**2)

#-------------------------------------------------------------------------------

def find_orientation(baz,SS,SZR,ERTR,ERRZ):

    """
    This function calculates the best back azimuth (phi) and sensor misorientation (orient) based on the 
    given quality criteria: signal strength (SS), similarity of vertical and radial components (SZR), 
    transverse-to-radial energy ratio (ERTR), and radial-to-vertical energy ratio (ERRZ).

    The cost function combines these criteria in such a way that maximizing the cost function helps to
    find the optimal back azimuth and corresponding orientation. The function outputs the best back azimuth, orientation,
    and the values of the quality criteria at the best azimuth index.

    Parameters:
    ----------
    baz : float
        Initial back azimuth value from the taup model (degrees).
    SS : np.array
        Array of signal strength values for each azimuth angle.
    SZR : np.array
        Array of similarity values between vertical and radial components for each azimuth angle.
    ERTR : np.array
        Array of transverse-to-radial energy ratios for each azimuth angle.
    ERRZ : np.array
        Array of radial-to-vertical energy ratios for each azimuth angle.

    Returns:
    -------
    phi : float
        The best back azimuth angle (degrees) that minimizes the cost function.
    orient : float
        The sensor misorientation angle (degrees), defined as the difference between the true back azimuth 
        and the estimated back azimuth.
    SS_best : float
        The signal strength value at the best azimuth.
    SZR_best : float
        The similarity between vertical and radial components at the best azimuth.
    ERTR_best : float
        The transverse-to-radial energy ratio at the best azimuth.
    ERRZ_best : float
        The radial-to-vertical energy ratio at the best azimuth.
    """
    
    # Find best index
    cost_function = (
                (1-SS) +  # Minimizing energy (equal to Maximizing (1 - transversal energy))
                SZR +     # Maximizing similarity
                ERTR +    # Maximizing transverse-to-radial ratio
                ERRZ      # Maximizing radial-to-vertical ratio
                )
    
    # Best index will maximize the cost function                    
    best_index = np.argmax(cost_function)
                            
    # Get azimuth and correct for angles above 360
    phi = round(ang[best_index])
    orient = round(baz_pair - ang[best_index])
                            
    # Get argument of maximum coherence:
    SS_best = SS[best_index]
    SZR_best = SZR[best_index]
    ERTR_best = ERTR[best_index]
    ERRZ_best = ERRZ[best_index]

    return phi,orient,SS_best,SZR_best,ERTR_best,ERRZ_best

# --------------------------------------------------------------------------

def Braunmiller_Pornsopin_algorithm(tr1,tr2,trZ,noise,ang,dphi,baz,CCVR_MIN=0.7,SNR_MIN=10,TRR_MIN=0.45,RVR_MIN=-1):

    """
    Estimate back azimuth using P-wave particle motion and apply quality criteria.

    This algorithm estimates the back azimuth by analyzing P-wave particle motion in an isotropic, 
    homogeneous layered medium. In such a medium, the P-wave energy propagates along a great circle 
    path between the source and receiver, with horizontal components defining the radial direction.
    The angle between the radial direction and true north gives the back azimuth. The P-wave energy 
    is confined to the vertical and radial components, with no energy in the transverse component.

    The sensor 'misorientation angle' is the difference between the true back azimuth (from the taup model) 
    and the empirically estimated back azimuth, with positive values representing a clockwise misorientation.

    This method applies several quality criteria to filter out unreliable results from component malfunctions or 
    missing horizontal components:
    
    == Quality criteria for automatic processing ==

    To select reliable back azimuths in automatic processing, the following five quality criteria are applied:
     - (1) Overall signal strength of the radial component.
     - (2) Similarity between the vertical and radial components.
     - (3) Transverse-to-radial energy ratio.
     - (4) Radial-to-vertical energy ratio.
     - (5) Signal-to-noise ratio (SNR) on the vertical component.

    The function uses these criteria to assess the quality of the estimated back azimuth, and classifies the result 
    as 'good' or 'bad' based on the user-defined thresholds.

    -----------  
    Parameters:
    ----------
    tr1 : np.array
        The first horizontal component of the seismogram.
    tr2 : np.array
        The second horizontal component of the seismogram.
    trZ : np.array
        The vertical component of the seismogram.
    noise : np.array
        Noise window to calculate the signal-to-noise ratio (SNR).
    ang : np.array
        Array of azimuth angles to search through (in degrees).
    dphi : float
        Step size for the azimuth search (in degrees).
    baz : float
        Back azimuth from the taup model (in degrees).
    CCVR_MIN : float, optional
        Minimum required similarity of vertical and radial components (default is 0.7).
    SNR_MIN : float, optional
        Minimum required signal-to-noise ratio (default is 10).
    TRR_MIN : float, optional
        Minimum required transverse-to-radial energy ratio (default is 0.7).
    RVR_MIN : float, optional
        Maximum allowed radial-to-vertical energy ratio (default is 0.2).
        
    
    Returns:
    -------
    dict
        A dictionary containing the following calculated quality criteria, estimated azimuth, and additional results:
        
        - 'phi' : float
            The estimated back azimuth angle (in degrees) based on the best-fit azimuth search.
        
        - 'baz' : float
            The true back azimuth (in degrees) from the taup model, which serves as a reference for comparison with the estimated azimuth.
        
        - 'SNR' : float
            The signal-to-noise ratio (SNR) of the vertical component, expressed in decibels (dB), representing the strength of the signal relative to noise.
        
        - 'Quality' : str
            A classification of the estimated azimuth quality ('good' or 'bad'), based on the comparison of various quality criteria and thresholds.
        
        - 'orient' : float
            The sensor misorientation angle (in degrees), representing the difference between the true back azimuth (from the taup model) and the empirically estimated azimuth.
        
        - 'SS_best' : float
            The best signal strength value for the optimal azimuth, quantifying the energy of the radial component for the best-fit azimuth.
        
        - 'SZR_best' : float
            The best similarity score between the vertical and radial components for the optimal azimuth, indicating how well the vertical and radial components align.
        
        - 'ERTR_best' : float
            The best transverse-to-radial energy ratio for the optimal azimuth, assessing the degree to which the transverse component contaminates the radial component.
        
        - 'ERRZ_best' : float
            The best radial-to-vertical energy ratio for the optimal azimuth, showing the relative strength of the radial component compared to the vertical component.
        
        - 'signal_strength' : np.array
            A NumPy array containing the signal strength values for each azimuth tested in the search range, reflecting the overall energy of the radial component.
        
        - 'similarity_ZR' : np.array
            A NumPy array containing the similarity (correlation) coefficients between the vertical and radial components for each azimuth tested.
        
        - 'energy_ratio_TR' : np.array
            A NumPy array containing the transverse-to-radial energy ratios for each azimuth tested, evaluating the amount of transverse energy relative to radial energy.
        
        - 'energy_ratio_RZ' : np.array
            A NumPy array containing the radial-to-vertical energy ratios for each azimuth tested, indicating how much radial energy is present compared to vertical energy.
    
    Notes:
    ------
    The algorithm assumes that the back azimuth is between 0 and 360 degrees and that the sensor 
    misorientation is defined as the difference between the true back azimuth and the empirically estimated back azimuth.
    """
    
    # Initialize quality criteria
    signal_strength = np.zeros(len(ang))
    similarity_ZR = np.zeros(len(ang))
    energy_ratio_TR = np.zeros(len(ang))
    energy_ratio_RZ = np.zeros(len(ang))
    
    # Search through azimuths and find best-fit azimuth
    for k, an in enumerate(ang):
        R, T = rotate_ne_rt(tr1, tr2, an)

        # (1) Overall signal strength of the transversal component
        signal_strength[k] = energy(T)

        # (2) Similarity of vertical and radial components
        similarity_ZR[k] = np.corrcoef(trZ, R)[0, 1]
        
        # (3) Transverse-to-radial energy ratio
        energy_ratio_TR[k] = 1-energy(T) / energy(R)
        
        # (4) Radial-to-vertical energy ratio
        energy_ratio_RZ[k] = 1-energy(R) / energy(trZ)  
    
    # (5) Signal-to-noise ratio on vertical component
    SNR = round(10.0 * np.log10(rms(trZ)**2 / rms(noise)**2), 1)

    # Normalizing the signal strength of the transversal component
    signal_strength = (signal_strength - np.min(signal_strength)) / (np.max(signal_strength) - np.min(signal_strength)) 

    phi,orient,SS_best,SZR_best,ERTR_best,ERRZ_best = find_orientation(baz,signal_strength,similarity_ZR,energy_ratio_TR,energy_ratio_RZ):

    if (SZR_best >= CCVR_MIN) & (SNR >= SNR_MIN) & (1-ERTR_best >= TRR_MIN) & (ERRZ_best <= RVR_MIN):
        quality = 'good'
    else:
        quality = 'bad'

    # Collect results
    results = {
        'phi': phi,
        'baz': baz,
        'SNR': SNR,
        'Quality': quality,
        'orient': orient,
        'SS_best': SS_best,
        'SZR_best': SZR_best,
        'ERTR_best': ERTR_best,
        'ERRZ_best': ERRZ_best,
        'signal_strength': signal_strength,
        'similarity_ZR': similarity_ZR,
        'energy_ratio_TR': energy_ratio_TR,
        'energy_ratio_RZ': energy_ratio_RZ,
    }
    
    return results

# ----------------------------------------------------------------------------------------------------
def plotting_event_orientation()
   
                
    # --------------------
    # figure 
    fig = plt.figure(figsize=(30, 10),constrained_layout=True)
    fig.suptitle('Evento: '+event_name+'(Δ: '+str(gcarc_pair_round)+'° | M: '+str(event.magnitudes[-1].mag)+' '+event.magnitudes[-1].magnitude_type+' | D: '+str(round(depth_event))+' km) \n BAZ: '+str(baz_pair_round)+'° | PHI: '+str(phi)+'° | orient: '+str(orient)+'° | SNR: '+str(SNR)+' dB',fontsize=20)

    # creating grid
    gs = fig.add_gridspec(1, 2,width_ratios=[3,1])

    gs0 = gs[0].subgridspec(3, 1)
    gs1 = gs[1].subgridspec(4, 1)
                                
    # Rotating components
    new_R, new_T = rotate_ne_rt(tr1_data_filtered, tr2_data_filtered, phi)

    # Transversal data
    ax1 = fig.add_subplot(gs0[0, 0])
    ax1.plot(trZ_time,new_T,'-k',lw=2)
    ax1.plot(trZ_signal_time,tr2,c='gray',ls='--',lw=0.5)
    ax1.annotate('Transversal', (0.9, 0.85),xycoords='axes fraction',fontsize=15, va='center',bbox=dict(boxstyle="round", fc="white"))
    ax1.set_xlim(-TIME_WINDOW,TIME_WINDOW)
    ax1.tick_params(axis="x", labelbottom=False)
    ax1.axvline(x=-TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
    ax1.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)

    # Radial data
    ax2 = fig.add_subplot(gs0[1, 0], sharex=ax1, sharey=ax1)
    ax2.plot(trZ_time,new_R,'-k')
    ax2.plot(trZ_signal_time,tr1,c='gray',ls='--',lw=0.5)
    ax2.annotate('Radial', (0.9, 0.85),xycoords='axes fraction',fontsize=15, va='center',bbox=dict(boxstyle="round", fc="white"))
    ax2.tick_params(axis="x", labelbottom=False)
    ax2.axvline(x=-TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
    ax2.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)

    # Vertical data and noise and signal window
    ax3 = fig.add_subplot(gs0[2, 0], sharex=ax1, sharey=ax1)
    ax3.plot(trZ_time,trZ_data_filtered,'-k')
    ax3.plot(trZ_noise_time,noise,'--b')
    ax3.plot(trZ_signal_time,trZ,'--r')
    ax3.annotate('Vertical', (0.9, 0.85),xycoords='axes fraction',fontsize=15, va='center',bbox=dict(boxstyle="round", fc="white"))
    ax3.set_xlabel('Timelag (s)',fontsize=15)
    ax3.axvline(x=-TIME_START_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)
    ax3.axvline(x=TIME_FINAL_P_REGIONAL, ymin=0, ymax=1,color='gray',linestyle='--',lw=1)

    # Transversal signal strength
    ax4 = fig.add_subplot(gs1[0, 0])
    ax4.plot(ang,SS,'.k')
    ax4.plot(phi,SS_max,'*r',ms=10)
    ax4.set_ylim(0,1)
    ax4.tick_params(axis="x", labelbottom=False)
    ax4.set_title('Transversal signal strength',fontsize=15)
                                
    # Similarity between vertical and radial
    ax5 = fig.add_subplot(gs1[1, 0], sharex=ax4)
    ax5.plot(ang,SZR,'.k')
    ax5.plot(phi,SZR_max,'*r',ms=10)
    ax5.set_ylim(0,1)
    ax5.set_xlim(0,360)
    ax5.tick_params(axis="x", labelbottom=False)
    ax5.set_title('Similarity between vertical and radial',fontsize=15)

    # Transverse-to-Radial Energy Ratio
    ax6 = fig.add_subplot(gs1[2, 0], sharex=ax5)
    ax6.plot(ang,ERTR,'.k')
    ax6.plot(phi,ERTR_max,'*r',ms=10)
    ax6.tick_params(axis="x", labelbottom=False)
    ax6.set_title('Transverse-to-Radial Energy Ratio',fontsize=15)
                                                                
    # Radial-to-Vertical Energy Ratio
    ax7 = fig.add_subplot(gs1[3, 0], sharex=ax6)
    ax7.plot(ang,ERRZ,'.k')
    ax7.plot(phi,ERRZ_max,'*r',ms=10)
    ax7.set_title('Radial-to-Vertical Energy Ratio',fontsize=15)
    ax7.set_xlabel('Orientation Angle (deg)',fontsize=15)

    # --------------------------
    # Adding global location map

    ax_map = plt.axes([0.0, 0.82, 0.15, 0.15], projection=ccrs.Orthographic(central_latitude=sta_lat,central_longitude=sta_lon))
    ax_map.set_global()

    # ---------------------
    # Adding background map 

    ax_map.add_feature(cfeature.LAND)
    ax_map.add_feature(cfeature.OCEAN)
    ax_map.add_feature(cfeature.COASTLINE)
                            
    ax_map.scatter(ev_lon,ev_lat,color="y",marker='*',s=200,ec='k',transform=ccrs.PlateCarree())
    ax_map.scatter(sta_lon,sta_lat,color="r",marker='^',s=50,transform=ccrs.PlateCarree())
    ax_map.plot([sta_lon, ev_lon], [sta_lat, ev_lat], c='gray',ls='-',lw=2, transform=ccrs.Geodetic())
                                            
    output_figure_ORIENTATION = ORIENTATION_OUTPUT+'ORIENTATION_FIGURES/EARTHQUAKES/'+sta+'/'
    os.makedirs(output_figure_ORIENTATION,exist_ok=True)
    fig.savefig(output_figure_ORIENTATION+'ORIENTATION_'+sta+'_'+event_name+'_'+label+'.png',dpi=100)
    plt.close()
                    

---
# Main program
---

In [19]:
print('===============')
print('Reading catalog')
print('===============')
print('\n')

cat = pd.read_csv(CATALOG_FILE)
cat.tail(2)

Reading catalog




,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
2156,2025-03-10T02:33:14.389Z,71.1971,-8.1925,10.000,6.5,mww,69.0,44.0,4.587,1.23,...,2025-03-11T02:57:16.212Z,"36 km NNE of Olonkinbyen, Svalbard and Jan Mayen",earthquake,7.78,1.899,0.041,58.0,reviewed,us,us
2157,2025-03-14T23:42:35.671Z,-55.7385,-27.1122,35.379,6.0,mww,71.0,68.0,5.589,0.91,...,2025-03-16T23:53:17.562Z,South Sandwich Islands region,earthquake,10.47,6.746,0.093,11.0,reviewed,us,us


In [22]:
print('================')
print('Reading stations')
print('================')
print('\n')

STATIONS_xml = sorted(glob.glob(XML_DIR+'*'))
print('Number of stations:',len(STATIONS_xml))

Reading stations


Number of stations: 88


In [23]:
STATIONS_xml

['/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.AQDB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.BB19B.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.BSCB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.BSFB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.C2SB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.CDSB.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.CLDB.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.CNLB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.CPSB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.DIAM.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.ESAR.3rd.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEARS/XML/BL.FRTB.nmx.dataless.xml',
 '/home/sysop/dados_posdoc/PROJETO_RSBR_15_YEAR

In [15]:
def calculate_orientation(sta):
    
    # ---------------------------
    # Retrieving events waveforms
    # ---------------------------
    
    for event in tqdm(cat, total=len(cat),desc=sta+' orientation'):
        # ------------------------------
        # Check if the event is eligible

        depth_event = event.origins[-1].depth/1000
        event_time = event.origins[-1].time
        event_name = event.origins[-1].time.strftime('%Y.%j.%H.%M.%S')

        year = event.origins[-1].time.strftime('%Y')
        julian_day = event.origins[-1].time.strftime('%j')

        network = 'ON'
        station = sta    
                   
        # ---------------
        # Import XML file
                        
        xml_file = glob.glob(XML_DIR+network+'.'+sta+'*')[0]
        station_xml = op.read_inventory(xml_file)
                            
        # Epicentral distance:
            
        sta_lon = station_xml[-1][-1][-1].longitude
        sta_lat = station_xml[-1][-1][-1].latitude
            
        ev_lat = event.origins[-1].latitude
        ev_lon = event.origins[-1].longitude
            
        dist_pair,az_pair,baz_pair = gps2dist_azimuth(ev_lat, ev_lon,sta_lat, sta_lon)
        gcarc_pair = kilometers2degrees(dist_pair/1000)
        baz_pair_round = round(baz_pair)
        gcarc_pair_round = round(gcarc_pair)
        dist_pair_round = round(dist_pair)  

        # Taup: theoretical travel times 
        model = TauPyModel(model="iasp91")
        arrivals = model.get_travel_times(source_depth_in_km=depth_event,distance_in_degree=gcarc_pair)
        
        # Event time + P arrival time
        event_time = event_time+arrivals[0].time
        
        if gcarc_pair > GCARC_MIN and gcarc_pair < GCARC_MAX:
            # ----------------------------
            # Check if feather file exists
            
            output_FEATHER_FILES_ORIENTATION = ORIENTATION_OUTPUT+'FEATHER_FILES/'+sta+'/'
            
            file_feather_name = output_FEATHER_FILES_ORIENTATION+sta+'_'+event_name+'_orientation_data.feather'
    
            station_pwd = glob.glob(EVENT_DIR+year+'/'+network+'/'+station+'/*')
    
            if os.path.isfile(file_feather_name):
                pass
        
            else:
                # -------------------------------
                # Check if components file exists
                        
                if (len([i for i in station_pwd if 'HHE.D' in i or 'HH2.D' in i]) > 0 and
                    len([i for i in station_pwd if 'HHN.D' in i or 'HH1.D' in i]) > 0 and
                    len([i for i in station_pwd if 'HHZ.D' in i]) > 0):
    
                    if (len(glob.glob([i for i in station_pwd if 'HHE.D' in i or 'HH2.D' in i][0]+'/*'+year+'.'+julian_day)) > 0 and
                        len(glob.glob([i for i in station_pwd if 'HHN.D' in i or 'HH1.D' in i][0]+'/*'+year+'.'+julian_day)) > 0 and
                        len(glob.glob([i for i in station_pwd if 'HHZ.D' in i][0]+'/*'+year+'.'+julian_day)) > 0):
                    
                        try:
                        
                            file_HHE = glob.glob([i for i in station_pwd if 'HHE.D' in i or 'HH2.D' in i][0]+'/*'+year+'.'+julian_day)[0]
                            file_HHN = glob.glob([i for i in station_pwd if 'HHN.D' in i or 'HH1.D' in i][0]+'/*'+year+'.'+julian_day)[0]
                            file_HHZ = glob.glob([i for i in station_pwd if 'HHZ.D' in i][0]+'/*'+year+'.'+julian_day)[0]
        
                            # --------
                            # Data HHE
                            
                            tr2_data_file = op.read(file_HHE)
                            tr2_data_file.trim(event_time-TIME_WINDOW,event_time+TIME_WINDOW)
                            tr2_data_file.taper(type='hann',max_percentage=0.1)
                            tr2_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            tr2_data_filtered = tr2_data_file[0].data
                
                            # --------
                            # Data HHN
                
                            tr1_data_file = op.read(file_HHN)
                            tr1_data_file.trim(event_time-TIME_WINDOW,event_time+TIME_WINDOW)
                            tr1_data_file.taper(type='hann',max_percentage=0.1)
                            tr1_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            tr1_data_filtered = tr1_data_file[0].data
                            
                            # --------
                            # Data HHZ
                            
                            trZ_data_file = op.read(file_HHZ)
                            trZ_data_file.trim(event_time-TIME_WINDOW,event_time+TIME_WINDOW)
                            trZ_data_file.taper(type='hann',max_percentage=0.1)
                            trZ_data_file.filter('bandpass',freqmin=PERIOD_BANDS[0],freqmax=PERIOD_BANDS[1],zerophase=True, corners=4)
                            trZ_data_filtered = trZ_data_file[0].data
                            trZ_time = trZ_data_file[0].times()-TIME_WINDOW       
                            
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Signal and noise windows
    
                            signal_window = (trZ_time >= -TIME_START_P_REGIONAL) & (trZ_time <= TIME_FINAL_P_REGIONAL)
                            noise_window = (trZ_time >= -(TIME_START_P_REGIONAL+TIME_FINAL_P_REGIONAL)) & (trZ_time <= -TIME_START_P_REGIONAL)
                            
                            noise = trZ_data_filtered[noise_window]
                            trZ_noise_time = trZ_time[noise_window]
    
                            tr2 = tr2_data_filtered[signal_window]
                            tr1 = tr1_data_filtered[signal_window]
                            trZ = trZ_data_filtered[signal_window]
                            trZ_signal_time = trZ_time[signal_window]
                                 
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Search Space of BAZ
                        
                            dphi = 0.1
                            ang = np.arange(0., 360., dphi)
                            
                            # -------------------------------------------------------------------------------------------------------------------------------
                            # Calculating the optimal orientation
                                            
                            results = Braunmiller_Pornsopin_algorithm(tr1,tr2,trZ,noise,ang,dphi,baz_pair_round,CCVR_MIN=0.7,SNR_MIN=10,TRR_MIN=0.45,RVR_MIN=-1):

                                           
                            # ----------------------------------------------------------------------------------------------------
                                                               
                                # ----------------------------------------------------------------------------------------------------
                                # Creating a Pandas DataFrame:
                                column_info = [sta,event_name,dist_pair,gcarc_pair,baz_pair,SS_max,SZR_max,ERTR_max,ERRZ_max,SNR,phi,orient,label]
                                columns_header = ['station','event','distance','gcarc','baz','signal_strength','similarity_vertical_radial','energy_transverse_radial','energy_radial_vertical','SNR','phi','orient','quality']
                                orient_p_wave_df = pd.DataFrame(column_info, index=columns_header).T
                        
                                # ----------------------------------------------------------------------------------------------------
                                # Convert from pandas to Arrow and saving in feather formart file
                                os.makedirs(output_FEATHER_FILES_ORIENTATION,exist_ok=True)
                                feather.write_feather(orient_p_wave_df, file_feather_name)

                        except:
                            pass


In [16]:
start_time = time.time()

with Pool(processes=20) as p:
    max_ = len(STATION_LST)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(calculate_orientation,STATION_LST):
            pbar.update()

print('\n')
print("--- %.2f execution time (min) ---" % ((time.time() - start_time)/60))
print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [06:54<00:00, 414.75s/it]



--- 6.92 execution time (min) ---


